# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [1]:
import os
import dgl
import json
import torch
import torch as th
import dgl.nn as dglnn
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv, SAGEConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:2


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [5]:
datasets = ['train', 'valid', 'test']
dataset_data = {}

for dataset_name in tqdm(datasets):
    file_path = f"../../data_processing/dgl/data_new/exp1-2/training_data/exp_1/secureBERT_50/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_1/secureBERT_50/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_1/secureBERT_50/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_1/secureBERT_50/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

# dataloaders = create_dataloaders(4)
dataloaders = create_dataloaders(16)

- Turn the print message to a log file

In [7]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GraphSAGE_secureBERT_50.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0911_23:14_GraphSAGE_secureBERT_50.log


### Model

In [8]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'mean')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'mean')  # Output layer

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.layer2(g, h)  # You can apply another non-linearity here if needed
        
        g.ndata['h'] = h
        hg = dgl.mean_nodes(g, 'h')
        return hg


- Model Forward  

In [9]:
def model_fn(data, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
#     labels = labels.float()

    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
#     logits = logits.mean(dim=1)
#     print(labels)
#     print(labels.shape)
#     print(logits)
#     print(logits.shape)

    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability
    preds = logits.argmax(1) # for GAT
#     print(preds)
#     preds = logits.argmax() # for graphSAGE
    
    '''
    问题出在GAT和GraphSAGE模型的输出形状上。
    在你的 GAT 模型中，最后一层的输出是一个一维的张量，因此 logits 是一个形状为 (batch_size,) 的张量。这是因为 GAT 模型最后一层的输出被处理成了一个节点的预测，而不是整个图的预测。
    在 GraphSAGE 模型中，最后一层的输出形状是 (num_nodes, out_dim)，也就是说，它返回了整个图的节点级别的预测。因此，logits 是一个二维的张量，形状为 (num_nodes, out_dim)。
    当你尝试在一个一维张量上调用 logits.argmax(1) 时，会产生维度错误，因为它期望一个二维张量来执行操作。
    所以，在 GraphSAGE 模型中，你应该将以下行：
    '''
    
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [10]:
seed = 8787
same_seeds(seed)

model = GraphSAGE(in_dim=50, hidden_dim=16, out_dim=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [11]:
model.layer1.fc_self.weight

Parameter containing:
tensor([[ 0.3966, -0.2622, -0.4172, -0.2120,  0.0392, -0.1415,  0.4194,  0.1208,
         -0.0180, -0.4136, -0.3660,  0.2784,  0.1991, -0.0898,  0.3969, -0.1612,
          0.2863, -0.3886, -0.0421, -0.1185, -0.3744,  0.3962,  0.2371, -0.3675,
         -0.4098,  0.3475,  0.1936,  0.1666, -0.3460,  0.3121, -0.2045, -0.0577,
          0.4003, -0.1058,  0.1000, -0.1319,  0.1262, -0.1793, -0.3533,  0.2065,
         -0.3789,  0.1936,  0.2644, -0.4193,  0.2918,  0.1618,  0.0928, -0.0824,
         -0.1039,  0.0728],
        [ 0.2606, -0.4032,  0.3426,  0.3319, -0.1503, -0.0656, -0.1955, -0.2699,
         -0.1863,  0.4212,  0.1890, -0.3355, -0.1435,  0.0435,  0.2325,  0.1970,
          0.0711,  0.3621,  0.0238,  0.4196,  0.2668,  0.4046, -0.0977, -0.2018,
          0.1634, -0.1156,  0.4119,  0.2383, -0.0069,  0.0162, -0.0569, -0.2738,
          0.2987, -0.0467,  0.3767, -0.4006, -0.1762, -0.3149,  0.3337,  0.2914,
         -0.3179,  0.1646,  0.2352,  0.1848, -0.0304, -0.23

- Check if model really load the model_dict

In [12]:
model = GraphSAGE(in_dim=50, hidden_dim=16, out_dim=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.layer1.fc_self.weight

Parameter containing:
tensor([[ 0.3966, -0.2622, -0.4172, -0.2120,  0.0392, -0.1415,  0.4194,  0.1208,
         -0.0180, -0.4136, -0.3660,  0.2784,  0.1991, -0.0898,  0.3969, -0.1612,
          0.2863, -0.3886, -0.0421, -0.1185, -0.3744,  0.3962,  0.2371, -0.3675,
         -0.4098,  0.3475,  0.1936,  0.1666, -0.3460,  0.3121, -0.2045, -0.0577,
          0.4003, -0.1058,  0.1000, -0.1319,  0.1262, -0.1793, -0.3533,  0.2065,
         -0.3789,  0.1936,  0.2644, -0.4193,  0.2918,  0.1618,  0.0928, -0.0824,
         -0.1039,  0.0728],
        [ 0.2606, -0.4032,  0.3426,  0.3319, -0.1503, -0.0656, -0.1955, -0.2699,
         -0.1863,  0.4212,  0.1890, -0.3355, -0.1435,  0.0435,  0.2325,  0.1970,
          0.0711,  0.3621,  0.0238,  0.4196,  0.2668,  0.4046, -0.0977, -0.2018,
          0.1634, -0.1156,  0.4119,  0.2383, -0.0069,  0.0162, -0.0569, -0.2738,
          0.2987, -0.0467,  0.3767, -0.4006, -0.1762, -0.3149,  0.3337,  0.2914,
         -0.3179,  0.1646,  0.2352,  0.1848, -0.0304, -0.23

In [13]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = GraphSAGE(in_dim=50, hidden_dim=16, out_dim=167)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_secureBERT_50.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=5e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)


criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

total_steps = 10

# save the best model
best_val_loss = float('inf')
patience = 10  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
#     scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for data in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

#         print(best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/8250 [00:00<?, ?it/s]

09/11/2023, 23:16:05# labels of 5000: tensor([ 49,  52,  31,  91,  46,  87, 163,  54,  93, 150, 143, 162, 146,  14,
        102,  24], device='cuda:2') torch.Size([16])
09/11/2023, 23:16:05# predicted of 5000: tensor([  0, 154,  86, 142,  46,   1,  71,  89,  48,   8,  71,  20, 146, 109,
        102,   8], device='cuda:2') torch.Size([16])
09/11/2023, 23:17:01# total batches: 8250
09/11/2023, 23:17:01# Epoch 0 | Train Loss: 3.5220 | Train Accuracy: 0.2974


Validation:   0%|          | 0/1032 [00:00<?, ?it/s]

09/11/2023, 23:17:01# labels of Validation: tensor([ 23, 118,   1,  90, 147,  92,  86, 109,  48,  67,  80, 146,  50,  71,
        103, 144], device='cuda:2') torch.Size([16])
09/11/2023, 23:17:01# predicted of Validation: tensor([ 23, 118, 141,  91, 147, 142,  86,  42,  74,  67,  80, 146,  50,  71,
        108,  81], device='cuda:2') torch.Size([16])
09/11/2023, 23:17:01# labels of 0: tensor([ 23, 118,   1,  90, 147,  92,  86, 109,  48,  67,  80, 146,  50,  71,
        103, 144], device='cuda:2') torch.Size([16])
09/11/2023, 23:17:01# predicted of 0: tensor([ 23, 118, 141,  91, 147, 142,  86,  42,  74,  67,  80, 146,  50,  71,
        108,  81], device='cuda:2') torch.Size([16])
09/11/2023, 23:17:14# labels of Validation: tensor([117,  78,   3,  73,  80,  67,   6,   3,  43, 104, 146, 161, 143,  39,
        160,  49], device='cuda:2') torch.Size([16])
09/11/2023, 23:17:14# predicted of Validation: tensor([117,  78,  79,  73,  80,  67,   6,   3,  43,  74, 146, 161, 141,  39,
        160,

...

09/11/2023, 23:37:42# labels of Validation: tensor([ 26, 136, 111, 166, 144,  92,  43,  78,  64,  54,  77, 124,  79, 100,
         46, 164], device='cuda:2') torch.Size([16])
09/11/2023, 23:37:42# predicted of Validation: tensor([ 26, 136,   4, 166,  12, 124,  43,  67,  68, 124,  77,  76,  35, 100,
         46, 124], device='cuda:2') torch.Size([16])
09/11/2023, 23:37:42# labels of 0: tensor([ 26, 136, 111, 166, 144,  92,  43,  78,  64,  54,  77, 124,  79, 100,
         46, 164], device='cuda:2') torch.Size([16])
09/11/2023, 23:37:42# predicted of 0: tensor([ 26, 136,   4, 166,  12, 124,  43,  67,  68, 124,  77,  76,  35, 100,
         46, 124], device='cuda:2') torch.Size([16])
09/11/2023, 23:37:56# labels of Validation: tensor([ 61, 152,  53,  58,  66,  90,  62,  14,  25,  11,  56,  75,  60,  94,
        130,   4], device='cuda:2') torch.Size([16])
09/11/2023, 23:37:56# predicted of Validation: tensor([ 61,  76, 124,  58,  66,  68,  62,  12,  25,  76,  56, 124,  94,  94,
        130,

Training:   0%|          | 0/8250 [00:00<?, ?it/s]

09/11/2023, 23:39:25# labels of 5000: tensor([  3,  13, 137, 145,  85, 158, 145,  70,  14,  91, 138,  70, 162,  53,
         26,  78], device='cuda:2') torch.Size([16])
09/11/2023, 23:39:25# predicted of 5000: tensor([  3,  13, 137, 145,  85, 143, 145,  70,  31, 108, 138, 149, 143, 143,
         26,  78], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:20# total batches: 8250
09/11/2023, 23:40:20# Epoch 9 | Train Loss: 1.5769 | Train Accuracy: 0.6013


Validation:   0%|          | 0/1032 [00:00<?, ?it/s]

09/11/2023, 23:40:20# labels of Validation: tensor([ 44, 101, 120, 148,  71,  94, 145, 166,  99, 103,  93, 152,  33, 164,
        136, 129], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:20# predicted of Validation: tensor([125, 101, 120, 148,  71,  94, 145, 166,  99, 114, 103, 125, 125, 125,
        136, 129], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:20# labels of 0: tensor([ 44, 101, 120, 148,  71,  94, 145, 166,  99, 103,  93, 152,  33, 164,
        136, 129], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:20# predicted of 0: tensor([125, 101, 120, 148,  71,  94, 145, 166,  99, 114, 103, 125, 125, 125,
        136, 129], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# labels of Validation: tensor([ 97,  43, 108, 134,  67,  67,  99,  78,  36, 116, 152,  31, 102,   7,
         62, 120], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# predicted of Validation: tensor([125,  43, 108, 134,  67,  67,  99,  67, 125, 125, 125,  81, 102,   7,
         62,

### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [14]:
# load the pretrained model
pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_secureBERT_50.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for data in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(data, model, criterion, device, count, which_type='test')
        labels = data[1].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# mapped_true_labels_np = np.array(mapped_true_labels)
# mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-secureBERT_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-secureBERT_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/1032 [00:00<?, ?it/s]

09/11/2023, 23:40:34# labels of Test: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# predicted of Test: tensor([ 47,  54,  12,  76, 124,  47,  74,  76, 124,  47,  31, 124,  12,  12,
         47,  54], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# labels of 0: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# predicted of 0: tensor([ 47,  54,  12,  76, 124,  47,  74,  76, 124,  47,  31, 124,  12,  12,
         47,  54], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# labels: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:34# predicted: tensor([ 47,  54,  12,  76, 124,  47,  74,  76, 124,  47,  31, 124,  12,  12,
         47,  54], device='cuda:2') torch.Size([16])
09/11/2023, 23:40:48# labels of Test: tensor([163, 163, 163, 163, 163, 163, 163, 163, 163, 163, 163, 163, 163, 163,
   

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/11/2023, 23:40:50# report path: classification_report/classification_report-secureBERT_50-graphSAGE-0.xlsx
09/11/2023, 23:40:50# label path: classification_report/mapped_true_predicted_labels-secureBERT_50-graphSAGE-0.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/11/2023, 23:40:50# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00       100
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00       100
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00       100
    T1003.002_7fa4ea18694f2552547b65e23952cabb       0.99      1.00      1.00       100
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.12      0.02      0.03       100
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00       100
        T1007_9d03c91bdae5a80f17f89c987942b5a8       0.98      0.94      0.96       100
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.00      0.00      0.00       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.18      0.09      0.12     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
